### Démarrage de tensorboard et imports principaux

In [1]:
# Agrandir le notebook ?
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

# Commande à entrer dans le prompt anaconda si on veut éviter que notre jupyter crash sur un long entrainement
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

# Démarrage de tensorboard pour notebook
%load_ext tensorboard


import sys
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
import datetime
import os
import pandas as pd
from PIL import Image
import shutil  
from math import ceil, floor

from tensorflow.python.framework import ops #pour tenter de reset tensorboard, sans grand succès
ops.reset_default_graph()

### Hyper paramètres


In [2]:
'''
# Parametres de verification : 

list_indiv_id = ['1', '2']
list_epochs = [1, 1]
list_batch_size = [100, 64]
list_nb_layers = [6, 2]
list_l1 = [0.01, 0]
list_l2 = [0.01, 0.001]
list_batch_norm = [0, 1]
list_dropout = [0, 0.2]
list_filters_per_layers = [64, 32]
list_filters_double = [2, 0]
list_MLP_end = [120, 0]
list_activation = ['relu', 'relu']
list_kernel = [(3,3), (3,3)]
list_padding = ['same', 'same']
list_max_or_avg_pool = ['max', 'avg']
list_learning_r = [0.01, 0.01]
list_momentum = [0.9, 0.85]
list_optimizer = ['SGD', 'Adam']

current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
main_directory = os.getcwd() + "\\logs\\convnets\\logs_" + current_time
'''

'\n# Parametres de verification : \n\nlist_indiv_id = [\'1\', \'2\']\nlist_epochs = [1, 1]\nlist_batch_size = [100, 64]\nlist_nb_layers = [6, 2]\nlist_l1 = [0.01, 0]\nlist_l2 = [0.01, 0.001]\nlist_batch_norm = [0, 1]\nlist_dropout = [0, 0.2]\nlist_filters_per_layers = [64, 32]\nlist_filters_double = [2, 0]\nlist_MLP_end = [120, 0]\nlist_activation = [\'relu\', \'relu\']\nlist_kernel = [(3,3), (3,3)]\nlist_padding = [\'same\', \'same\']\nlist_max_or_avg_pool = [\'max\', \'avg\']\nlist_learning_r = [0.01, 0.01]\nlist_momentum = [0.9, 0.85]\nlist_optimizer = [\'SGD\', \'Adam\']\n\ncurrent_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")\nmain_directory = os.getcwd() + "\\logs\\convnets\\logs_" + current_time\n'

In [3]:
'''
#Real Test 1 -> importance des layers (refaire le test 5 mais sans la regularization)
# CONSTANTES : filters = 64, batch_size = 50, epochs = 30 (30 car avec beaucoup de layers ça va être long),
#lr = 0.001, momentum = 0.9, optimizer Adam, padding = same, maxpool, relu, kernel = (3,3), 
#avec regu L1L2 à 0 et dropout à 0 (pas de dropout), (sans Batchnorm, sans MLP final, sans filters double)

#6 NB layers fixes -> 8, 16, 32, 64, 96, 128

# LEXIQUE PARAM : 
# * filters_double permet de savoir toutes les combien de couche on double les filtres, si 0 on double pas

list_indiv_id = ['1', '2', '3', '4', '5', '6']
list_epochs = [30, 30, 30, 30, 30, 30]
list_batch_size = [50, 50, 50, 50, 50, 50]
list_nb_layers = [8,16,32,64,96,128]
list_l1 = [0, 0, 0, 0, 0, 0]
list_l2 = [0, 0, 0, 0, 0, 0]
list_batch_norm = [0, 0, 0, 0, 0, 0]
list_dropout = [0, 0, 0, 0, 0, 0]
list_filters_per_layers = [64, 64, 64, 64, 64, 64]
list_filters_double = [0, 0, 0, 0, 0, 0]
list_MLP_end = [0, 0, 0, 0, 0, 0]
list_activation = ['relu','relu','relu','relu','relu','relu']
list_kernel = [(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)]
list_padding = ['same','same','same','same','same','same']
list_max_or_avg_pool = ['max','max','max','max','max','max']
list_learning_r = [0.001,0.001,0.001,0.001,0.001,0.001]
list_momentum = [0.9,0.9,0.9,0.9,0.9,0.9]
list_optimizer = ['Adam','Adam','Adam','Adam','Adam','Adam']

current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
main_directory = os.getcwd() + "\\logs\\convnets\\logs_" + current_time
'''

'\n#Real Test 1 -> importance des layers (refaire le test 5 mais sans la regularization)\n# CONSTANTES : filters = 64, batch_size = 50, epochs = 30 (30 car avec beaucoup de layers ça va être long),\n#lr = 0.001, momentum = 0.9, optimizer Adam, padding = same, maxpool, relu, kernel = (3,3), \n#avec regu L1L2 à 0 et dropout à 0 (pas de dropout), (sans Batchnorm, sans MLP final, sans filters double)\n\n#6 NB layers fixes -> 8, 16, 32, 64, 96, 128\n\n# LEXIQUE PARAM : \n# * filters_double permet de savoir toutes les combien de couche on double les filtres, si 0 on double pas\n\nlist_indiv_id = [\'1\', \'2\', \'3\', \'4\', \'5\', \'6\']\nlist_epochs = [30, 30, 30, 30, 30, 30]\nlist_batch_size = [50, 50, 50, 50, 50, 50]\nlist_nb_layers = [8,16,32,64,96,128]\nlist_l1 = [0, 0, 0, 0, 0, 0]\nlist_l2 = [0, 0, 0, 0, 0, 0]\nlist_batch_norm = [0, 0, 0, 0, 0, 0]\nlist_dropout = [0, 0, 0, 0, 0, 0]\nlist_filters_per_layers = [64, 64, 64, 64, 64, 64]\nlist_filters_double = [0, 0, 0, 0, 0, 0]\nlist_MLP_e

In [4]:
'''
#Real Test 2 -> importance des filters
# CONSTANTES : nb_layers = 8, batch_size = 50, epochs = 50, lr = 0.001, momentum = 0.9, optimizer Adam, 
#padding = same, maxpool, relu, kernel = (3,3), MLP avec 128 neuronnes, pas de regularization
# 
# * 5 convnets avec filters fixes
#   - 1 sans filters double avec filters (16)
#   - 1 sans filters double avec filters (32)
#   - 1 sans filters double avec filters (64)
#   - 1 sans filters double avec filters (96)
#   - 1 sans filters double avec filters (128)

# * 5 convnets avec filters evolutifs (filter double sur mes blocs de 2 couches de convolution)
#   - 1 filters double (2) avec filters (16)  (va faire 16, 32, 64, 128)
#   - 1 filters double (2) avec filters (32)  (va faire 32, 64, 128, 256)
#   - 1 filters double (4) avec filters (16)  (va faire 16, 16, 32, 32)
#   - 1 filters double (4) avec filters (32)  (va faire 32, 32, 64, 64)
#   - 1 filters double (4) avec filters (64)  (va faire 64, 64, 128, 128)

list_indiv_id = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
list_epochs = [50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
list_batch_size = [50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
list_nb_layers = [8,8,8,8,8,8,8,8,8,8]
list_l1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
list_l2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
list_batch_norm = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
list_dropout = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
list_filters_per_layers = [16, 32, 64, 96, 128, 16, 32, 16, 32, 64]
list_filters_double = [0, 0, 0, 0, 0, 2, 2, 4, 4, 4]
list_MLP_end = [128, 128, 128, 128, 128, 128, 128, 128, 128, 128]
list_activation = ['relu','relu','relu','relu','relu','relu','relu','relu','relu','relu']
list_kernel = [(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)]
list_padding = ['same','same','same','same','same','same','same','same','same','same']
list_max_or_avg_pool = ['max','max','max','max','max','max','max','max','max','max']
list_learning_r = [0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001]
list_momentum = [0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9]
list_optimizer = ['Adam','Adam','Adam','Adam','Adam','Adam','Adam','Adam','Adam','Adam']

current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
main_directory = os.getcwd() + "\\logs\\convnets\\logs_" + current_time
'''

'\n#Real Test 2 -> importance des filters\n# CONSTANTES : nb_layers = 8, batch_size = 50, epochs = 50, lr = 0.001, momentum = 0.9, optimizer Adam, \n#padding = same, maxpool, relu, kernel = (3,3), MLP avec 128 neuronnes, pas de regularization\n# \n# * 5 convnets avec filters fixes\n#   - 1 sans filters double avec filters (16)\n#   - 1 sans filters double avec filters (32)\n#   - 1 sans filters double avec filters (64)\n#   - 1 sans filters double avec filters (96)\n#   - 1 sans filters double avec filters (128)\n\n# * 5 convnets avec filters evolutifs (filter double sur mes blocs de 2 couches de convolution)\n#   - 1 filters double (2) avec filters (16)  (va faire 16, 32, 64, 128)\n#   - 1 filters double (2) avec filters (32)  (va faire 32, 64, 128, 256)\n#   - 1 filters double (4) avec filters (16)  (va faire 16, 16, 32, 32)\n#   - 1 filters double (4) avec filters (32)  (va faire 32, 32, 64, 64)\n#   - 1 filters double (4) avec filters (64)  (va faire 64, 64, 128, 128)\n\nlist_indiv

In [5]:
'''
#Real Test 2 -> importance des filters (UNIQUEMENT FILTERS 16 FIXE)
# CONSTANTES : nb_layers = 8, batch_size = 50, epochs = 50, lr = 0.001, momentum = 0.9, optimizer Adam, 
#padding = same, maxpool, relu, kernel = (3,3), MLP avec 128 neuronnes, pas de regularization
# 
# * 5 convnets avec filters fixes
#   - 1 sans filters double avec filters (16)
#   - 1 sans filters double avec filters (32)
#   - 1 sans filters double avec filters (64)
#   - 1 sans filters double avec filters (96)
#   - 1 sans filters double avec filters (128)

# * 5 convnets avec filters evolutifs (filter double sur mes blocs de 2 couches de convolution)
#   - 1 filters double (2) avec filters (16)  (va faire 16, 32, 64, 128)
#   - 1 filters double (2) avec filters (32)  (va faire 32, 64, 128, 256)
#   - 1 filters double (4) avec filters (16)  (va faire 16, 16, 32, 32)
#   - 1 filters double (4) avec filters (32)  (va faire 32, 32, 64, 64)
#   - 1 filters double (4) avec filters (64)  (va faire 64, 64, 128, 128)

list_indiv_id = ['1']
list_epochs = [3]
list_batch_size = [50]
list_nb_layers = [7]
list_l1 = [0]
list_l2 = [0]
list_batch_norm = [0]
list_dropout = [0]
list_filters_per_layers = [16]
list_filters_double = [0]
list_MLP_end = [128]
list_activation = ['relu']
list_kernel = [(3,3)]
list_padding = ['same']
list_max_or_avg_pool = ['max']
list_pool_frequency = [3]
list_learning_r = [0.001]
list_momentum = [0.9]
list_optimizer = ['Adam']

current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
main_directory = os.getcwd() + "\\logs\\convnets\\logs_" + current_time
'''

'\n#Real Test 2 -> importance des filters (UNIQUEMENT FILTERS 16 FIXE)\n# CONSTANTES : nb_layers = 8, batch_size = 50, epochs = 50, lr = 0.001, momentum = 0.9, optimizer Adam, \n#padding = same, maxpool, relu, kernel = (3,3), MLP avec 128 neuronnes, pas de regularization\n# \n# * 5 convnets avec filters fixes\n#   - 1 sans filters double avec filters (16)\n#   - 1 sans filters double avec filters (32)\n#   - 1 sans filters double avec filters (64)\n#   - 1 sans filters double avec filters (96)\n#   - 1 sans filters double avec filters (128)\n\n# * 5 convnets avec filters evolutifs (filter double sur mes blocs de 2 couches de convolution)\n#   - 1 filters double (2) avec filters (16)  (va faire 16, 32, 64, 128)\n#   - 1 filters double (2) avec filters (32)  (va faire 32, 64, 128, 256)\n#   - 1 filters double (4) avec filters (16)  (va faire 16, 16, 32, 32)\n#   - 1 filters double (4) avec filters (32)  (va faire 32, 32, 64, 64)\n#   - 1 filters double (4) avec filters (64)  (va faire 64

In [16]:
#Real Test 3 -> importance du MLP avant la softmax
# CONSTANTES : nb_layers = 8, batch_size = 50, epochs = 100, lr = 0.001, momentum = 0.9, optimizer Adam, 
#padding = same, maxpool, relu, kernel = (3,3), filters_per_layer = 64 filters_double = 0, pas de regularization
# 

# * 1 convent sans MLP

# * 5 convnets avec MLP
#   - 1 avec (MLP 32)
#   - 1 avec (MLP 64)
#   - 1 avec (MLP 128)
#   - 1 avec (MLP 256)
#   - 1 avec (MLP 512)


list_indiv_id = ['1', '2', '3', '4', '5', '6']
list_epochs = [5, 5, 5, 5, 5, 5]
list_batch_size = [50, 50, 50, 50, 50, 50]
list_nb_layers = [8,8,8,8,8,8]
list_l1 = [0, 0, 0, 0, 0, 0]
list_l2 = [0, 0, 0, 0, 0, 0]
list_batch_norm = [0, 0, 0, 0, 0, 0]
list_dropout = [0, 0, 0, 0, 0, 0]
list_filters_per_layers = [64, 64, 64, 64, 64, 64]
list_filters_double = [0, 0, 0, 0, 0, 0]
list_MLP_end = [0, 32, 64, 128, 256, 512]
list_activation = ['relu','relu','relu','relu','relu','relu']
list_kernel = [(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)]
list_padding = ['same','same','same','same','same','same']
list_max_or_avg_pool = ['max','max','max','max','max','max']
list_pool_frequency = [2,2,2,2,2,2]
list_learning_r = [0.001,0.001,0.001,0.001,0.001,0.001]
list_momentum = [0.9,0.9,0.9,0.9,0.9,0.9]
list_optimizer = ['Adam','Adam','Adam','Adam','Adam','Adam']

current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
main_directory = os.getcwd() + "\\logs\\convnets\\logs_" + current_time

### Fonctions pour préparer le dataset

In [17]:
# Chargement des datasets de train et de validation + one hot encoding
def load_dataset():
    # Chargement des données cifar10
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode encoding sur les labels
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

# Normalisation pour accroître la vitesse du modèle (en redimensionnant les pixels)
def prep_pixels(train, test):
    # Convertion des int en float
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # Normalisation pour avoir des nombres entre 0 et 1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # Retourner les images normalisées
    return train_norm, test_norm


### Classe Python pour définir les individus

In [18]:
# Classe pour les convnets
class IndividuConvnets:
    def __init__(self, indiv_id='1', epochs=10, batch_size=1, nb_layers=2, l1=0, l2=0, batch_norm=0, dropout=0, filters_per_layers=64, filters_double=6, MLP_end=0, activation='relu', kernel=(3,3), padding='same', max_or_avg_pool=0, pool_frequency=2, learning_r=0.01, momentum=0.9, optimizer='SGD'):
        # Initialisation de nos variables
        self.time_fit = datetime.datetime.now()
        self.my_reguralizer = None
        
        if nb_layers < 2:
            self.nb_layers = 2
        else:
            self.nb_layers = nb_layers
            
        self.loss = 0
        self.accuracy = 0
        self.indiv_id = indiv_id
        self.epochs = epochs
        self.batch_size = batch_size

        self.l1 = l1
        self.l2 = l2

        self.batch_norm = batch_norm
        self.dropout = dropout
        self.filters_per_layers = filters_per_layers
        
        # Création d'un variable qui va garder la valeur de filters_per_layers (elle changera dans le modèle)
        self.keep_filters_per_layers = filters_per_layers
        
        
        # filters_double doit être supérieur ou égal à 2 OU égal à 0
        if filters_double < 2:
            self.filters_double = 0
        else : 
            self.filters_double = filters_double
        
        if MLP_end < 0:
            self.MLP_end = 0
        else:
            self.MLP_end = MLP_end
        
        self.activation = activation
        self.kernel = kernel
        self.padding = padding
        self.max_or_avg_pool = max_or_avg_pool
        self.pool_frequency = pool_frequency
        self.learning_r = learning_r
        self.momentum = momentum
        self.optimizer = optimizer
    
    # ToString()
    def __str__(self):
        ma_liste = []
        ma_liste.append("indiv_id:{},\n ".format(self.indiv_id))
        ma_liste.append("epochs:{},\n ".format(self.epochs))
        ma_liste.append("batch_size:{},\n ".format(self.batch_size))
        ma_liste.append("nb_layers:{},\n ".format(self.nb_layers))
        ma_liste.append("l1:{},\n ".format(self.l1))
        ma_liste.append("l2:{},\n ".format(self.l2))
        ma_liste.append("batch_norm:{},\n ".format(self.batch_norm))
        ma_liste.append("dropout:{},\n ".format(self.dropout))
        ma_liste.append("filters_per_layers:{},\n ".format(self.filters_per_layers))
        ma_liste.append("filters_double:{},\n ".format(self.filters_double))
        ma_liste.append("MLP_end:{},\n ".format(self.MLP_end))
        ma_liste.append("activation:{},\n ".format(self.activation))
        ma_liste.append("kernel:\n ")
        ma_liste.append("{},\n ".format(self.kernel))
        ma_liste.append("padding:{},\n ".format(self.padding))
        ma_liste.append("max_or_avg_pool:{}\n".format(self.max_or_avg_pool))
        ma_liste.append("pool_frequency:{}\n".format(self.pool_frequency))
        ma_liste.append("learning_r:{}\n".format(self.learning_r))
        ma_liste.append("momentum:{}\n".format(self.momentum))
        ma_liste.append("optimizer:{}\n".format(self.optimizer))
            
        return ma_liste
    
    # (Modele 2 conv + norm ? + pool) * X -> MLP -> softmax sortie 10 -> MODELE BLOC 2
    # D'autres modeles seront crees par la suite
    def create_and_train_model(self, trainX, trainY, testX, testY, main_directory):
        start = datetime.datetime.now()
        
        # Choix d'un emplacement pour les logs
        log_dir=main_directory+"\\log_"+self.indiv_id+"\\tensorboard_data\\"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        print("log dir = ",log_dir)
        
        # l1 et l2
        if self.l1 > 0 and self.l2 > 0:
            self.my_regularizer = regularizers.l1_l2(l1=self.l1 / self.nb_layers, l2=self.l2 / self.nb_layers)
        if self.l1 > 0:
            self.my_regularizer = regularizers.l1(self.l1 / self.nb_layers)
        elif self.l2 > 0:
            self.my_regularizer = regularizers.l2(self.l2 / self.nb_layers)
        else:
            self.my_reguralizer = None
            
        # Definir notre modèle basique
        model = Sequential()

        # Faire toutes les convs nécessaires (conv * 2 + max pool)
        counter_filters_double = 0 # Var pour doubler les filtres
        counter_pool_freq = 0 # var pour savoir où placer les couches de pooling
        counter_pool = 0 # var pour nommer les max / avg pool
        
        for i in range(0, self.nb_layers):
            
            print("counter_pool_freq = ", counter_pool_freq)
            # Traitement pour doubler les filtres
            if counter_filters_double >= self.filters_double and self.filters_double != 0:
                self.filters_per_layers = self.filters_per_layers * 2
                print("filters = ", self.filters_per_layers)
                counter_filters_double = 0
            
            # Première conv, on fixe l'input shape
            if i == 0:
                model.add(Conv2D(self.filters_per_layers, self.kernel, activation=self.activation, kernel_regularizer=self.my_reguralizer, padding=self.padding,input_shape=(32, 32, 3), name='conv_'+str(self.filters_per_layers)+'_'+str(i+1)))
            else:
                # Couche de conv + rajouts selon nos hyperparams
                model.add(Conv2D(self.filters_per_layers, self.kernel, activation=self.activation, kernel_regularizer=self.my_reguralizer, padding=self.padding, name='conv_'+str(self.filters_per_layers)+'_'+str(i+1)))
            
            # Après avoir créé une conv on incrémente nos compteurs (sauf counter_pool)
            counter_filters_double = counter_filters_double + 1
            counter_pool_freq = counter_pool_freq + 1
            
            # Ajouts de la regularization / du pooling selon les hyperparamètres saisis
            
            if self.batch_norm == 1:
                    model.add(BatchNormalization( name='batchnorm_'+str(i+1)))
            
            if self.pool_frequency == counter_pool_freq:    
                #go max ou avg pooling
                if self.max_or_avg_pool == 'max':
                    model.add(MaxPooling2D((2, 2), padding='same', name='max_pool_'+str(counter_pool+1)))
                    counter_pool = counter_pool + 1
                else:
                    model.add(AveragePooling2D((2, 2), padding='same', name='avg_pool_'+str(counter_pool+1)))
                    counter_pool = counter_pool + 1
                counter_pool_freq = 0        
        
        # Fin des convs -> neural network classique
        model.add(Flatten(name='Flatten'))
        
        #tTrain dans un MLP avant la fin si on le souhaite
        if self.MLP_end > 0:
            model.add(Dense(128, activation='relu', kernel_regularizer=self.my_reguralizer, name='MLP_'+str(self.MLP_end)))

            #mettre dropout sur les Dense, pas opti sur les convnets (mais on peut le faire pour le démontrer ??)
            if self.dropout > 0:
                model.add(Dropout(self.dropout))
        
        #notre output
        model.add(Dense(10, activation='softmax', name='output')) 

        # Compiler le modele
        if self.optimizer == 'SGD':
            print("SGD, learning_r = ", self.learning_r, " momentum = ", self.momentum, "\n")
            opt = SGD(lr=self.learning_r, momentum=self.momentum)
        else:
            print("Adam learning_r = ", self.learning_r, " momentum = ", self.momentum, "\n")
            opt = Adam(lr=self.learning_r, beta_1=self.momentum) # beta_1 => notation pour momentum Adam
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Entrainer le modele
        history = model.fit(trainX, trainY, epochs=self.epochs, batch_size=self.batch_size, validation_data=(testX, testY), verbose=1, callbacks=[tensorboard_callback])
        
        # Garder une trace du temps nécessaire pour fit (peut être pas la meilleure méthode)
        end = datetime.datetime.now()
        self.time_fit = end - start
        print("\nTime for fit = ", round(self.time_fit.total_seconds(),2)) # Round avec total_seconds()
        
        #Arpès que le fit soit fait, remettre filters_per_layers à sa valeur initiale pour un meilleur log 
        self.filters_per_layers = self.keep_filters_per_layers

        return history, model
    
    
    def save_model(self, history, model, main_directory, current_time):
        
        # Sauvegarde du modèle
        plot_model(model, "model.png")
        
        # Deplacement modele au bon endroit
        shutil.move(os.getcwd()+"\\model.png", main_directory+"\\log_"+self.indiv_id+"\\model.png")
        
        # Afficher nos résultats dans un graphique matplotlib sauvegardé
        pyplot.gcf().subplots_adjust(hspace = 0.5)

        # Afficher la loss
        pyplot.subplot(211)
        pyplot.title('Cross Entropy Loss')
        pyplot.plot(history.history['loss'], color='blue', label='train')
        pyplot.plot(history.history['val_loss'], color='orange', label='test')
        
        # Afficher l'accuracy
        pyplot.subplot(212)
        pyplot.title('Classification Accuracy')
        pyplot.plot(history.history['accuracy'], color='blue', label='train')
        pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
        
        # Sauvegarde
        filename = main_directory+"\\log_"+self.indiv_id+"\\"
        pyplot.savefig(filename + 'plot.png')
        pyplot.close()
       
        
        print("LOSS : ", round(history.history['loss'][self.epochs-1].item(), 3))
        print("VAL_LOSS : ", round(history.history['val_loss'][self.epochs-1].item(), 3))
        print("ACCURACY : ", round(history.history['accuracy'][self.epochs-1].item(), 3))
        print("VAL_ACCURACY : ", round(history.history['val_accuracy'][self.epochs-1].item(), 3))
        
        # attributs pour créer les csv indivudels et le csv global
        self.loss = round(history.history['loss'][self.epochs-1].item(), 3)
        self.val_loss = round(history.history['val_loss'][self.epochs-1].item(), 3)
        self.accuracy = round(history.history['accuracy'][self.epochs-1].item(), 3)
        self.val_accuracy = round(history.history['val_accuracy'][self.epochs-1].item(), 3)
        self.time_taken = round(self.time_fit.total_seconds(),2)
        
        # Créer un dataframe pandas (avec hyperparams) et le sauvegarder en CSV
        df = pd.DataFrame({'indiv_id': [self.indiv_id],
                           'epochs': [self.epochs],
                           'batch_size': [self.batch_size],
                           'nb_layers': [self.nb_layers],
                           'l1': [self.l1],
                           'l2': [self.l2],
                           'batch_norm': [self.batch_norm],
                           'dropout': [self.dropout],
                           'filters_per_layers': [self.filters_per_layers],
                           'filters_double': [self.filters_double],
                           'MLP_end': [self.MLP_end],
                           'activation': [self.activation],
                           'kernel': [self.kernel],
                           'padding': [self.padding],
                           'max_or_avg_pool': [self.max_or_avg_pool],
                           'pool_frequency': [self.pool_frequency],
                           'loss': [self.loss],
                           'val_loss': [self.val_loss],
                           'accuracy': [self.accuracy],
                           'val_accuracy': [self.val_accuracy],
                           'time_taken' : [self.time_taken],
                           'learning_r' : [self.learning_r],
                           'momentum' : [self.momentum],
                           'optimizer' : [self.optimizer]
                          })
        
        df.to_csv(path_or_buf=filename+"recap.csv",index=False)
    
    # Lance toutes les étapes
    def exec_indiv(self, main_directory, current_time):
        
        # Charger les données
        trainX, trainY, testX, testY = load_dataset()
        
        # Normaliser les données
        trainX, testX = prep_pixels(trainX, testX)
        
        print("TrainX shape = ",np.shape(trainX))
        print("TestX shape = ",np.shape(testX), "\n")
        # Créer et entrainer le modele
        history, model = self.create_and_train_model(trainX, trainY, testX, testY, main_directory)
        
        # Sauvegarder le modèle
        save = self.save_model(history, model, main_directory, current_time)

### Classe Python qui va démarrer les tests des neural nets


In [19]:
# Classe générale qui va nous servir à effectuer des actions sur des individus
class MyTraining:
    # Prends un ID et une liste d'individus 
    def __init__(self, id_train, indiv_list):
        
        self.id_train = id_train
        self.indiv_list = indiv_list
    
    def train(self, main_directory, current_time):
        
        print("Start training\n")
        
        for indiv in self.indiv_list:
            print("indiv ", indiv.indiv_id, "\n")
            indiv.exec_indiv(main_directory, current_time)
            print("-----------------------------------------------------------------\n")
        
        # Fusion des csv 
        merge_csv = pd.DataFrame(columns=['indiv_id', 'epochs', 'nb_layers', 'l1', 'l2', 'batch_norm', 'dropout',
                                          'filters_per_layers', 'filters_double', 'MLP_end', 'activation', 'kernel',
                                          'padding','max_or_avg_pool', 'pool_frequency', 'loss', 'val_loss', 'accuracy', 'val_accuracy',
                                          'time_taken','learning_r', 'momentum', 'optimizer'])
        
        for indiv in self.indiv_list:
            merge_csv = merge_csv.append(
                             {'indiv_id': indiv.indiv_id, 'epochs': indiv.epochs, 'batch_size': indiv.batch_size,
                              'nb_layers' : indiv.nb_layers,'l1' : indiv.l1, 'l2' : indiv.l2, 'batch_norm': indiv.batch_norm,
                              'dropout' : indiv.dropout,'filters_per_layers' : indiv.filters_per_layers,
                              'filters_double' : indiv.filters_double,'MLP_end' : indiv.MLP_end,'activation' : indiv.activation,
                              'kernel' : indiv.kernel,'padding' : indiv.padding, 'max_or_avg_pool' : indiv.max_or_avg_pool,
                              'pool_frequency' : indiv.pool_frequency, 'loss' : indiv.loss,'val_loss' : indiv.val_loss,
                              'accuracy' : indiv.accuracy, 'val_accuracy' : indiv.val_accuracy,'time_taken' : indiv.time_taken,
                              'learning_r' : indiv.learning_r,'momentum': indiv.momentum, 'optimizer' : indiv.optimizer
                             },ignore_index=True)
        
        # sauvegarde
        merge_csv.to_csv(main_directory+"\\combined_recap.csv", index=False)
            
    
    def all_indiv(self):
        
        # Affiche les caractéristiques de l'ensemble des individus
        for indiv in self.indiv_list:
            print('\n'.join(indiv.__str__()))
            for tir in range(80): print('-', end='')
            print()

### Traitement général (train de l'ensemble des modèles)

In [20]:
# Création des individus (des neurals nets, ici convnet)
list_indiv = []
for num in range(len(list_indiv_id)):
    list_indiv.append(IndividuConvnets(
        list_indiv_id[num],
          list_epochs[num],
          list_batch_size[num],
          list_nb_layers[num],
          list_l1[num],
          list_l2[num],
          list_batch_norm[num],
          list_dropout[num],
          list_filters_per_layers[num],
          list_filters_double[num],
          list_MLP_end[num],
          list_activation[num],
          list_kernel[num],
          list_padding[num],
          list_max_or_avg_pool[num],
          list_pool_frequency[num],
          list_learning_r[num],
          list_momentum[num],
          list_optimizer[num]
        )
    )

# Chargement de la classe training, affichag
training_1 = MyTraining(1, list_indiv)
training_1.all_indiv()
training_1.train(main_directory, current_time)

indiv_id:1,
 
epochs:5,
 
batch_size:50,
 
nb_layers:8,
 
l1:0,
 
l2:0,
 
batch_norm:0,
 
dropout:0,
 
filters_per_layers:64,
 
filters_double:0,
 
MLP_end:0,
 
activation:relu,
 
kernel:
 
(3, 3),
 
padding:same,
 
max_or_avg_pool:max

pool_frequency:2

learning_r:0.001

momentum:0.9

optimizer:Adam

--------------------------------------------------------------------------------
indiv_id:2,
 
epochs:5,
 
batch_size:50,
 
nb_layers:8,
 
l1:0,
 
l2:0,
 
batch_norm:0,
 
dropout:0,
 
filters_per_layers:64,
 
filters_double:0,
 
MLP_end:32,
 
activation:relu,
 
kernel:
 
(3, 3),
 
padding:same,
 
max_or_avg_pool:max

pool_frequency:2

learning_r:0.001

momentum:0.9

optimizer:Adam

--------------------------------------------------------------------------------
indiv_id:3,
 
epochs:5,
 
batch_size:50,
 
nb_layers:8,
 
l1:0,
 
l2:0,
 
batch_norm:0,
 
dropout:0,
 
filters_per_layers:64,
 
filters_double:0,
 
MLP_end:64,
 
activation:relu,
 
kernel:
 
(3, 3),
 
padding:same,
 
max_or_avg_poo

TrainX shape =  (50000, 32, 32, 3)
TestX shape =  (10000, 32, 32, 3) 

log dir =  C:\Users\arnau\Desktop\quatrième_année\Deep_Learning\Projet_cifar-10\logs\convnets\logs_2020-02-03-10-17\log_5\tensorboard_data\
counter_pool_freq =  0
counter_pool_freq =  1
counter_pool_freq =  0
counter_pool_freq =  1
counter_pool_freq =  0
counter_pool_freq =  1
counter_pool_freq =  0
counter_pool_freq =  1
Adam learning_r =  0.001  momentum =  0.9 

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 29s 580us/sample - loss: 1.6860 - accuracy: 0.3711 - val_loss: 1.3054 - val_accuracy: 0.5276
Epoch 2/5
50000/50000 [==============================] - 26s 528us/sample - loss: 1.1694 - accuracy: 0.5790 - val_loss: 1.0109 - val_accuracy: 0.6358
Epoch 3/5
50000/50000 [==============================] - 28s 565us/sample - loss: 0.9335 - accuracy: 0.6665 - val_loss: 0.9556 - val_accuracy: 0.6628
Epoch 4/5
50000/50000 [==============================] - 26s 

### Partie tensorboard

In [21]:
# Procedure pour utiliser tensorboard
#  1 load la première cell
#  2 utiliser la derniere cell avec --logdir (précisez bien votre répertoire, plus sur que ça
#    fonctionne avec une string "mon_path"
#  3 Vous NE POURREZ PLUS update tensorboard sur ce port et il y aura des bugs, pour éviter ça
#    quand vous voulez faire une update, fermez jupyter notebook (shutdown total) et réouvrez le 
#    OU, faites kernel->interrupt et changez de port + de folder de log

#si vous voulez tenter de tuer des process
#os.system("taskkill /im tensorboard.exe /f") #kill tous les processus qui utilisent tensorboard
#os.system('!kill 18776') #kill le processus X

In [ ]:
# Liste des ports utilisés par tensorboard, attention ça se remplit vite et il faut kill jupyter pour clean
from tensorboard import notebook
notebook.list()

In [ ]:
# Code pour démarrer tensorboard dans le dossier souhaité [PRECISEZ BIEN LE DOSSIER ICI]
%tensorboard --logdir "CONVNETS_20200119-0243\logs_20200119-093909\tensorboard_data" --port=6066

In [ ]:
# Si vous avez la folie des grandeurs
notebook.display(port=6066, height=1000) 

In [ ]:
### Fichier CSV combined_recap + Graphique

In [ ]:
# Commandes pandas utiles
data_csv = pd.read_csv(main_directory + "\\combined_recap.csv")
#data_csv = pd.read_csv("C:\\Users\\arnau\\Desktop\\quatrième_année\\Deep_Learning\\Projet_cifar-10\\CONVNETS_20200119-2043\\combined_recap.csv")
data.head()
#meilleure accuracy, moins pire loss par ex
#data_csv.sort_values(["elapsed"], axis=0, 
                 #ascending=[False], inplace=True) 

# Afficher uniquement certaines colonnes
#dataX = data_csv.filter(items=['elapsed', 'label'])

#récupérer uniquement où la loss est < à X et ou kernel = (3,3) par exemple
#dataX = data_csv.loc[(data_csv['elapsed'] > 700) & (data_csv['threadName'].str.contains('Thread Group 1-2'))]
#dataX

#pd.set_option('display.max_rows', data3.shape[0]+1) #nombre de row max à afficher
#data_csv = pd.read_csv(main_directory+"\\logs_20200116-204456\\recap.csv")
#data_csv.head()

In [ ]:
image = pyplot.imread(main_directory + "\\logs_20200119-093909\\plot.png")
#image = pyplot.imread("C:\\Users\\arnau\\Desktop\\quatrième_année\\Deep_Learning\\Projet_cifar-10\\logs_20200119-093909\\plot.png")
pyplot.imshow(image)